# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
display(toxic_comments['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64

**Вывод**  
Классы несбалансированы. Необходимо учитывать это при обучении моделей.

Напишем функцию для лемматизации и очистки текста

In [ ]:
"""# функция лемматизации и удаления лишних символов
m = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())
"""

'# функция лемматизации и удаления лишних символов\nm = WordNetLemmatizer()\n\ndef lemmatize_text(text):\n    text = text.lower()\n    lemm_text = "".join(m.lemmatize(text))\n    cleared_text = re.sub(r\'[^a-zA-Z]\', \' \', lemm_text) \n    return " ".join(cleared_text.split())\n'

In [ ]:
# функция лемматизации и удаления лишних символов

def clear_text(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    return re_list

m = WordNetLemmatizer()

def lemmatize_text(text):
    word_list = nltk.word_tokenize(text)
    
    return ' '.join([m.lemmatize(w) for w in word_list])


In [ ]:
toxic_comments['text'] = toxic_comments['text'].apply(clear_text)

In [ ]:
toxic_comments['lemm_text'] = toxic_comments['text'].apply(lemmatize_text)

In [ ]:
toxic_comments

,text,toxic,lemm_text
0,Explanation Why the edits made under my userna...,0,Explanation Why the edits made under my userna...
1,D'aww He matches this background colour I'm se...,0,D'aww He match this background colour I 'm see...
2,Hey man I'm really not trying to edit war It's...,0,Hey man I 'm really not trying to edit war It ...
3,More I can't make any real suggestions on impr...,0,More I ca n't make any real suggestion on impr...
4,You sir are my hero Any chance you remember wh...,0,You sir are my hero Any chance you remember wh...
...,...,...,...
159566,And for the second time of asking when your vi...,0,And for the second time of asking when your vi...
159567,You should be ashamed of yourself That is a ho...,0,You should be ashamed of yourself That is a ho...
159568,Spitzer Umm theres no actual article for prost...,0,Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...


In [ ]:
toxic_comments = toxic_comments.drop(['text'], axis=1)

Создадим корпус из лематизированных и очищеных тексов

In [ ]:
corpus = toxic_comments['lemm_text'].values

Разобьем выборки

In [ ]:
features = corpus
target = toxic_comments['toxic'].values

train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=12345)

In [ ]:
stopwordss = set(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words = stopwordss)
train_features = count_tf_idf.fit_transform(train_features)


In [ ]:
test_features = count_tf_idf.transform(test_features)

## Обучение

### LogisticRegression

In [ ]:
lr_model = LogisticRegression()
hyperparams = [{'C':[10],   # так же подбирал [0.1, 1, 3]
                'class_weight':['balanced']}]
clf = GridSearchCV(lr_model, hyperparams, scoring='f1',cv=3)
clf.fit(train_features, train_target)
print("Лучшие параметры модели:")
print()
LR_best_params = clf.best_params_
print(LR_best_params)
print()
print('F1:', clf.best_score_)

Лучшие параметры модели:

{'C': 10, 'class_weight': 'balanced'}

F1: 0.762750967078337


### CatBoostClassifier 

In [ ]:
train_featurescat =  train_features.toarray() 
valid_featurescat =  valid_features.toarray() 
test_featurescat = test_features.toarray() 
cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=100, 
                                   max_depth=6, 
                                   learning_rate=0.9, 
                                   random_state=43)
cat_model.fit(train_featurescat, train_target, verbose=20)
print("Лучшие параметры модели:")
print()
CAT_best_params = clf.best_params_
print(CAT_best_params)
print()
print('F1:', clf.best_score_)

'\ntrain_featurescat =  train_features.toarray() \nvalid_featurescat =  valid_features.toarray() \ntest_featurescat = test_features.toarray() \ncat_model = CatBoostClassifier(eval_metric="F1", \n                                   iterations=100, \n                                   max_depth=6, \n                                   learning_rate=0.9, \n                                   random_state=43)\ncat_model.fit(train_featurescat, train_target, verbose=20)\nprint("Лучшие параметры модели:")\nprint()\nCAT_best_params = clf.best_params_\nprint(CAT_best_params)\nprint()\nprint(\'F1:\', clf.best_score_)'

F1: 0.7601224906430758   

### LightGBM model

Закомитил LightGBM model потому что в тренажере обрабатываются больше часа

In [ ]:
%%time

# подбирал параметры кросс валидацией

LightGBM_model = LGBMClassifier()
hyperparams = [{'max_depth' : [-1], # -1, 1
                'learning_rate':[0.1], # 0.03, 0.1, 0.3
                'n_estimators' : [1000],  # 200, 500, 1000
                'random_state':[12345]}]
clf = GridSearchCV(LightGBM_model, hyperparams, scoring='f1',cv=3)
clf.fit(train_features, train_target)
print("Лучшие параметры модели:")
print()
LGBM_best_params = clf.best_params_
print(LGBM_best_params)
print()
print('F1:', clf.best_score_)

'%%time\n\n# подбирал параметры кросс валидацией\n\nLightGBM_model = LGBMClassifier()\nhyperparams = [{\'max_depth\' : [-1], # -1, 1\n                \'learning_rate\':[0.1], # 0.03, 0.1, 0.3\n                \'n_estimators\' : [1000],  # 200, 500, 1000\n                \'random_state\':[12345]}]\nclf = GridSearchCV(LightGBM_model, hyperparams, scoring=\'f1\',cv=3)\nclf.fit(train_features, train_target)\nprint("Лучшие параметры модели:")\nprint()\nLGBM_best_params = clf.best_params_\nprint(LGBM_best_params)\nprint()\nprint(\'F1:\', clf.best_score_)'


F1: 0.7679683884224614  
Wall time: 9min 24s

## Выводы

### LogisticRegression

In [ ]:
%%time
lr_model = LogisticRegression()
lr_model.set_params(**LR_best_params)
lr_model.fit(train_features, train_target)
prediction = lr_model.predict(test_features)
f1 = f1_score(test_target, prediction)
print('F1 регрессии:', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))
print()

F1 регрессии: 0.7625858123569795

Матрица ошибок
[[27589  1087]
 [  573  2666]]

CPU times: user 19.8 s, sys: 14.7 s, total: 34.5 s
Wall time: 34.5 s


### CatBoostClassifier

In [ ]:
cat_model = LogisticRegression()
cat_model.set_params(**CAT_best_params)
cat_model.fit(train_features, train_target)
prediction = cat_model.predict(test_features)
f1 = f1_score(test_target, prediction)
print('F1', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))
print()

"\ncat_model = LogisticRegression()\ncat_model.set_params(**CAT_best_params)\ncat_model.fit(train_features, train_target)\nprediction = cat_model.predict(test_features)\nf1 = f1_score(test_target, prediction)\nprint('F1', f1)\nprint()\nprint('Матрица ошибок')\nprint(confusion_matrix(test_target, prediction))\nprint()"

F1 CatBoost: 0.7551954913702007

Матрица ошибок  
[[14191   170]  
 [  525  1072]]

### LightGBM model

In [ ]:
LightGBM_model = LogisticRegression()
LightGBM_model.set_params(**LGBM_best_params)
LightGBM_model.fit(train_features, train_target)
prediction = LightGBM_model.predict(test_features)
f1 = f1_score(test_target, prediction)
print('F1:', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))
print()

"\nLightGBM_model = LogisticRegression()\nLightGBM_model.set_params(**LGBM_best_params)\nLightGBM_model.fit(train_features, train_target)\nprediction = LightGBM_model.predict(test_features)\nf1 = f1_score(test_target, prediction)\nprint('F1:', f1)\nprint()\nprint('Матрица ошибок')\nprint(confusion_matrix(test_target, prediction))\nprint()"


F1 регрессии: 0.7600950118764845  

Матрица ошибок  
[[13730   617]  
 [  248  1363]]

**Вывод**
Все модели имеют удовлетворительное значение F1. Лучше всех справилась LightGBM model. Но у LogisticRegression меньше ложно отрицательных предсказаний. Значит меньше токсичных коментариев пройдут мимо модерации. То что у LogisticRegression болльше ложно положительных предсказаний не страшно. Так как после модерации они вернуться обратно.

Такрим образом, для бизнеса, лучше подходит модель LogisticRegression